In [208]:
import pandas as pd

In [209]:
train  = pd.read_csv("train_dataset_train.csv")

In [210]:
test = pd.read_csv("test_dataset_test.csv")

In [211]:
train.loc[train["Год_Поступления"] == 2212, "Год_Поступления"] = 2012

In [212]:
train.loc[train["Пол"] == "муж", "Пол"] = "Муж"
test.loc[test["Пол"] == "жен", "Пол"] = "Жен"
train.loc[train["Пол"].isna(), "Пол"] = "None"
test.loc[test["Пол"].isna(), "Пол"] = "None"

In [213]:
train["Изучаемый_Язык"] = train["Изучаемый_Язык"].fillna("None")
test["Изучаемый_Язык"] = test["Изучаемый_Язык"].fillna("None")

train.loc[train["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Английский" in x)), "Изучаемый_Язык"] = "Английский"
train.loc[train["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Англиийский" in x)), "Изучаемый_Язык"] = "Английский"
train.loc[train["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Немецкий" in x)), "Изучаемый_Язык"] = "Немецкий"
train.loc[train["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Французский" in x)), "Изучаемый_Язык"] = "Французский"

test.loc[test["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Английский" in x)), "Изучаемый_Язык"] = "Английский"
test.loc[test["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Англиийский" in x)), "Изучаемый_Язык"] = "Английский"
test.loc[test["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Немецкий" in x)), "Изучаемый_Язык"] = "Немецкий"
test.loc[test["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Французский" in x)), "Изучаемый_Язык"] = "Французский"

In [214]:
def parse_date(x):
    year = int(x.split('-')[0])
    month = x.split('-')[1]
    if(month[0]=="0"):
        month = month[1]
        
    month =  int(month)
    
    return year*12 + month
    
train["Дата_Рождения"] = train["Дата_Рождения"].apply(parse_date)
test["Дата_Рождения"] = test["Дата_Рождения"].apply(parse_date)

In [215]:
train["Год_Окончания_УЗ"] = train["Год_Окончания_УЗ"].fillna(-1)
test["Год_Окончания_УЗ"] = test["Год_Окончания_УЗ"].fillna(-1)

In [216]:
train["gape"] = train["Год_Поступления"].values - train["Год_Окончания_УЗ"].values
test["gape"] = test["Год_Поступления"].values - test["Год_Окончания_УЗ"].values

In [217]:
def parse_post(x):
    return x*12 + 9
    
train["gape_age"] = train["Год_Поступления"].fillna(-1).apply(parse_post).values - train["Дата_Рождения"].values
test["gape_age"] = test["Год_Поступления"].fillna(-1).apply(parse_post).values - test["Дата_Рождения"].values

In [218]:
train["Пособие"] = train["Пособие"].fillna(-1)
test["Пособие"] = test["Пособие"].fillna(-1)

In [219]:
def find_lcsubstr(s1, s2):
    m = [[0 for i in range(len(s2) + 1)] for j in range(len(s1) + 1)]  # Сгенерировать матрицу 0, чтобы облегчить последующие вычисления, на один столбец больше, чем длина строки
    mmax = 0  # Длина самого длинного совпадения
    p = 0  # Самое длинное совпадение соответствует последнему биту в s1
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i] == s2[j]: # Если равно, добавить существующую общую подстроку
                m[i + 1][j + 1] = m[i][j] + 1
                if m[i + 1][j + 1] > mmax:
                    mmax = m[i + 1][j + 1]
                    p = i + 1
    return s1[p - mmax:p], mmax

names = ["Страна_ПП", "Страна_Родители", "Город_ПП", "Регион_ПП", "Где_Находится_УЗ", "Уч_Заведение"]

for i in range(len(names)-1):
    for j in range(i + 1, len(names)):
        v = []
        for x, y in zip(train[names[i]], train[names[j]]):
            if pd.notnull(x) and pd.notnull(y):
                #print(i, j, x,y)
                val = find_lcsubstr(x, y)[1]/min(len(x), len(y))
                v.append(val)
            else:
                v.append(-1)

    train[names[i] + '_' + names[j] + "_diff"] = v

In [220]:
names = ["Страна_ПП", "Страна_Родители", "Город_ПП", "Регион_ПП", "Где_Находится_УЗ", "Уч_Заведение"]

for i in range(len(names)-1):
    for j in range(i + 1, len(names)):
        v = []
        for x, y in zip(test[names[i]], test[names[j]]):
            if pd.notnull(x) and pd.notnull(y):
                val = find_lcsubstr(x, y)[1]/min(len(x), len(y))
                v.append(val)
            else:
                v.append(-1)

    test[names[i] + '_' + names[j] + "_diff"] = v

In [221]:
train["Страна_ПП"] = train["Страна_ПП"].fillna("None")
test["Страна_ПП"] = test["Страна_ПП"].fillna("None")

train["Страна_ПП"] = train["Страна_ПП"].apply(lambda x: x.lower())
test["Страна_ПП"] = test["Страна_ПП"].apply(lambda x: x.lower())


train.loc[train["Страна_ПП"].apply(lambda x: ("казах" in x)), "Страна_ПП"] = "казах"
train.loc[train["Страна_ПП"].apply(lambda x: ("таджик" in x)), "Страна_ПП"] = "таджик"
train.loc[train["Страна_ПП"].apply(lambda x: ("кырг" in x)or("кирг" in x)), "Страна_ПП"] = "кырг"
train.loc[train["Страна_ПП"].apply(lambda x: ("росс" in x)), "Страна_ПП"] = "росс"

test.loc[test["Страна_ПП"].apply(lambda x: ("казах" in x)), "Страна_ПП"] = "казах"
test.loc[test["Страна_ПП"].apply(lambda x: ("таджик" in x)), "Страна_ПП"] = "таджик"
test.loc[test["Страна_ПП"].apply(lambda x: ("кырг" in x)or("кирг" in x)), "Страна_ПП"] = "кырг"
test.loc[test["Страна_ПП"].apply(lambda x: ("росс" in x)), "Страна_ПП"] = "росс"

In [222]:
train["Общежитие"] = train["Общежитие"].fillna(-1)
test["Общежитие"] = test["Общежитие"].fillna(-1)

train["Наличие_Матери"] = train["Наличие_Матери"].fillna(-1)
test["Наличие_Матери"] = test["Наличие_Матери"].fillna(-1)

train["Наличие_Отца"] = train["Наличие_Отца"].fillna(-1)
test["Наличие_Отца"] = test["Наличие_Отца"].fillna(-1)


In [223]:
train["Страна_Родители"] = train["Страна_Родители"].fillna("None")


train["Страна_Родители"] = train["Страна_Родители"].apply(lambda x: x.lower())


train.loc[train["Страна_Родители"].apply(lambda x: ("казах" in x)), "Страна_Родители"] = "казах"
train.loc[train["Страна_Родители"].apply(lambda x: ("таджик" in x)), "Страна_Родители"] = "таджик"
train.loc[train["Страна_Родители"].apply(lambda x: ("кырг" in x)or("кирг" in x)), "Страна_Родители"] = "кырг"
train.loc[train["Страна_Родители"].apply(lambda x: ("росс" in x)), "Страна_Родители"] = "росс"
train.loc[train["Страна_Родители"].apply(lambda x: ("кнр" in x)), "Страна_Родители"] = "китай"


test["Страна_Родители"] = test["Страна_Родители"].fillna("None")


test["Страна_Родители"] = test["Страна_Родители"].apply(lambda x: x.lower())


test.loc[test["Страна_Родители"].apply(lambda x: ("казах" in x)), "Страна_Родители"] = "казах"
test.loc[test["Страна_Родители"].apply(lambda x: ("таджик" in x)), "Страна_Родители"] = "таджик"
test.loc[test["Страна_Родители"].apply(lambda x: ("кырг" in x)or("кирг" in x)), "Страна_Родители"] = "кырг"
test.loc[test["Страна_Родители"].apply(lambda x: ("росс" in x)), "Страна_Родители"] = "росс"
test.loc[test["Страна_Родители"].apply(lambda x: ("кнр" in x)), "Страна_Родители"] = "китай"


In [224]:
train["Иностранец"] = train["Иностранец"].fillna(-1)
test["Иностранец"] = test["Иностранец"].fillna(-1)

train["Опекунство"] = train["Опекунство"].fillna(-1)
test["Опекунство"] = test["Опекунство"].fillna(-1)

train["Село"] = train["Село"].fillna(-1)
test["Село"] = test["Село"].fillna(-1)

In [225]:
from collections import Counter
ct = Counter()
for st in train[~train["Уч_Заведение"].isna()]["Уч_Заведение"]:
    #print(x)
    st = st.lower()
    rez = ''.join([x if(x.isalpha()or x.isdigit()) else ' ' for x in st])
    ct += Counter( rez.split() ) 

filtered_words = set([word for word, freq in ct.most_common() if freq>20])
words2id  = {word:i for i, word in enumerate(filtered_words)}
id2word = {v:k for k,v in words2id.items()}

def transform_place(st, words2id):
    x = np.zeros((len(words2id), ))
    
    if pd.notnull(st):
        st = st.lower()
        rez = ''.join([x if(x.isalpha() or x.isdigit()) else ' ' for x in st]).split()
        for w in rez:
            if w in words2id:
                x[words2id[w]] += 1
    return x

In [226]:
import numpy as np
A = np.vstack(train["Уч_Заведение"].apply(lambda x: transform_place(x, words2id)).tolist())
uchzavdf = pd.DataFrame(A, columns = [id2word[i]+"_учзав" for i in range(len(filtered_words))])

In [227]:
A = np.vstack(test["Уч_Заведение"].apply(lambda x: transform_place(x, words2id)).tolist())
uchzavdf_test = pd.DataFrame(A, columns = [id2word[i]+"_учзав" for i in range(len(filtered_words))])

In [228]:
def cityf(st):
    r = ''
    if pd.notnull(st):
    
        for w in st.split(' '):
            if(len(w)==0):
                continue
            if(w[0].isupper()):
                r += w
        
    if(len(r) == 0):
        r = 'none'
    return r

train["Город_ПП"] = train["Город_ПП"].apply(cityf)
test["Город_ПП"] = test["Город_ПП"].apply(cityf)

In [229]:
train["Регион_ПП"] = train["Регион_ПП"].fillna("none")
train["Регион_ПП"] = train["Регион_ПП"].apply(lambda a: a.lower())

test["Регион_ПП"] = test["Регион_ПП"].fillna("none")
test["Регион_ПП"] = test["Регион_ПП"].apply(lambda a: a.lower())

train["reg_pp"] = train["Регион_ПП"].copy()
test["reg_pp"] = test["Регион_ПП"].copy()


ss = set([ "алтайс",
 "казах",
 "тыва",
 "кемер", 
  "саха",
  "новосиб",
  "павлодар",
 "алтайс",
 "краснояр",
  "жалал",
 "хатлон",
 "казах",
 "алмат",
 "ляонин",
 "хайнань",
 "хэйлун",
 "ямало",
  "томск",
  "цзилинь",
  "иркут",
  "бадах",
  "иркут",
  "ханты"])


train.loc[train["Регион_ПП"].apply(lambda x: ("алтай" in x)), "Регион_ПП"] = "алтайс"
train.loc[train["Регион_ПП"].apply(lambda x: ("казах" in x)), "Регион_ПП"] = "казах"
train.loc[train["Регион_ПП"].apply(lambda x: ("тыва" in x)), "Регион_ПП"] = "тыва"
train.loc[train["Регион_ПП"].apply(lambda x: ("кемер" in x)), "Регион_ПП"] = "кемер"
train.loc[train["Регион_ПП"].apply(lambda x: ("саха" in x)), "Регион_ПП"] = "саха"
train.loc[train["Регион_ПП"].apply(lambda x: ("новосиб" in x)), "Регион_ПП"] = "новосиб"
train.loc[train["Регион_ПП"].apply(lambda x: ("павлодар" in x)), "Регион_ПП"] = "павлодар"
train.loc[train["Регион_ПП"].apply(lambda x: ("алай" in x)), "Регион_ПП"] = "алтайс"
train.loc[train["Регион_ПП"].apply(lambda x: ("краснояр" in x)), "Регион_ПП"] = "краснояр"
train.loc[train["Регион_ПП"].apply(lambda x: ("жалал" in x)), "Регион_ПП"] = "жалал"
train.loc[train["Регион_ПП"].apply(lambda x: ("хатлон" in x)), "Регион_ПП"] = "хатлон"
train.loc[train["Регион_ПП"].apply(lambda x: ("вко" in x)), "Регион_ПП"] = "казах"
train.loc[train["Регион_ПП"].apply(lambda x: ("алмат" in x)), "Регион_ПП"] = "алмат"
train.loc[train["Регион_ПП"].apply(lambda x: ("ляонин" in x)), "Регион_ПП"] = "ляонин"
train.loc[train["Регион_ПП"].apply(lambda x: ("хайнань" in x)), "Регион_ПП"] = "хайнань"
train.loc[train["Регион_ПП"].apply(lambda x: ("хэйлун" in x)), "Регион_ПП"] = "хэйлун"
train.loc[train["Регион_ПП"].apply(lambda x: ("ямало" in x)), "Регион_ПП"] = "ямало"
train.loc[train["Регион_ПП"].apply(lambda x: ("томск" in x)), "Регион_ПП"] = "томск"
train.loc[train["Регион_ПП"].apply(lambda x: ("цзилинь" in x)), "Регион_ПП"] = "цзилинь"
train.loc[train["Регион_ПП"].apply(lambda x: ("иркут" in x)), "Регион_ПП"] = "иркут"
train.loc[train["Регион_ПП"].apply(lambda x: ("бадах" in x)), "Регион_ПП"] = "бадах"
train.loc[train["Регион_ПП"].apply(lambda x: ("иркут" in x)), "Регион_ПП"] = "иркут"
train.loc[train["Регион_ПП"].apply(lambda x: ("ханты" in x)), "Регион_ПП"] = "ханты"
train.loc[train["Регион_ПП"].apply(lambda x: ("омск" in x)), "Регион_ПП"] = "омск"

train.loc[~train["Регион_ПП"].isin(ss), "Регион_ПП"] = "rare"


test.loc[test["Регион_ПП"].apply(lambda x: ("алтай" in x)), "Регион_ПП"] = "алтайс"
test.loc[test["Регион_ПП"].apply(lambda x: ("казах" in x)), "Регион_ПП"] = "казах"
test.loc[test["Регион_ПП"].apply(lambda x: ("тыва" in x)), "Регион_ПП"] = "тыва"
test.loc[test["Регион_ПП"].apply(lambda x: ("кемер" in x)), "Регион_ПП"] = "кемер"
test.loc[test["Регион_ПП"].apply(lambda x: ("саха" in x)), "Регион_ПП"] = "саха"
test.loc[test["Регион_ПП"].apply(lambda x: ("новосиб" in x)), "Регион_ПП"] = "новосиб"
test.loc[test["Регион_ПП"].apply(lambda x: ("павлодар" in x)), "Регион_ПП"] = "павлодар"
test.loc[test["Регион_ПП"].apply(lambda x: ("алай" in x)), "Регион_ПП"] = "алтайс"
test.loc[test["Регион_ПП"].apply(lambda x: ("краснояр" in x)), "Регион_ПП"] = "краснояр"
test.loc[test["Регион_ПП"].apply(lambda x: ("жалал" in x)), "Регион_ПП"] = "жалал"
test.loc[test["Регион_ПП"].apply(lambda x: ("хатлон" in x)), "Регион_ПП"] = "хатлон"
test.loc[test["Регион_ПП"].apply(lambda x: ("вко" in x)), "Регион_ПП"] = "казах"
test.loc[test["Регион_ПП"].apply(lambda x: ("алмат" in x)), "Регион_ПП"] = "алмат"
test.loc[test["Регион_ПП"].apply(lambda x: ("ляонин" in x)), "Регион_ПП"] = "ляонин"
test.loc[test["Регион_ПП"].apply(lambda x: ("хайнань" in x)), "Регион_ПП"] = "хайнань"
test.loc[test["Регион_ПП"].apply(lambda x: ("хэйлун" in x)), "Регион_ПП"] = "хэйлун"
test.loc[test["Регион_ПП"].apply(lambda x: ("ямало" in x)), "Регион_ПП"] = "ямало"
test.loc[test["Регион_ПП"].apply(lambda x: ("томск" in x)), "Регион_ПП"] = "томск"
test.loc[test["Регион_ПП"].apply(lambda x: ("цзилинь" in x)), "Регион_ПП"] = "цзилинь"
test.loc[test["Регион_ПП"].apply(lambda x: ("иркут" in x)), "Регион_ПП"] = "иркут"
test.loc[test["Регион_ПП"].apply(lambda x: ("бадах" in x)), "Регион_ПП"] = "бадах"
test.loc[test["Регион_ПП"].apply(lambda x: ("иркут" in x)), "Регион_ПП"] = "иркут"
test.loc[test["Регион_ПП"].apply(lambda x: ("ханты" in x)), "Регион_ПП"] = "ханты"
test.loc[test["Регион_ПП"].apply(lambda x: ("омск" in x)), "Регион_ПП"] = "омск"

test.loc[~test["Регион_ПП"].isin(ss), "Регион_ПП"] = "rare"


In [230]:
train["Где_Находится_УЗ"] = train["Где_Находится_УЗ"].fillna("none")
train["Где_Находится_УЗ"] = train["Где_Находится_УЗ"].apply(lambda a:a.split(",")[-1]).apply(cityf)

test["Где_Находится_УЗ"] = test["Где_Находится_УЗ"].fillna("none")
test["Где_Находится_УЗ"] = test["Где_Находится_УЗ"].apply(lambda a:a.split(",")[-1]).apply(cityf)

In [231]:
train["reg_pp"] = train["reg_pp"].fillna("none")
test["reg_pp"] = test["reg_pp"].fillna("none")


train["Уч_Заведение"] = train["Уч_Заведение"].fillna("none")
test["Уч_Заведение"] = test["Уч_Заведение"].fillna("none")

In [232]:
df = pd.concat([train, test], axis=0)
count_name = "Код_группы_count"
name = "Код_группы"
work_count = df[[name]].copy()
work_count[count_name] = 1
work_count = work_count.groupby(name).sum().reset_index()

In [233]:
sg = set(df[df["СрБаллАттестата"]>5]["Код_группы"])&set(df[df["СрБаллАттестата"]<=5]["Код_группы"])
a = np.zeros((len(train), ))
a[train["Код_группы"].isin(sg)] = 1
train["Strange_disp"] = a

a = np.zeros((len(test), ))
a[test["Код_группы"].isin(sg)] = 1
test["Strange_disp"] = a

In [234]:
a = np.array(list(set(df["Код_группы"].values)))
a.sort()
b = np.hstack([np.array([-1]), a[1:] - a[:-1]])
work_diff = pd.DataFrame.from_dict({"Код_группы":a, "diff_group":b})

In [235]:
a = np.array(list(set(df["Код_группы"].values)))
a.sort()
b = np.hstack([a[1:] - a[:-1], np.array([-1])])
work_diff_inv = pd.DataFrame.from_dict({"Код_группы":a, "diff_group_inv":b})

In [236]:
import numpy as np
a = np.array(list(set(df["ID"].values)))
a.sort()
b = np.hstack([np.array([-1]), a[1:] - a[:-1]])
dfid = pd.DataFrame.from_dict({"ID":a, "diff_ID":b})

In [237]:
import numpy as np
a = np.array(list(set(df["ID"].values)))
a.sort()
b = np.hstack([a[1:] - a[:-1], np.array([-1])])
dfidinv = pd.DataFrame.from_dict({"ID":a, "diff_ID_ind":b})

In [238]:
cat_feat = ["Пол", "Основания", "Изучаемый_Язык", "Пособие", 
            "Страна_ПП", "Общежитие", "Наличие_Матери", "Наличие_Отца", "Страна_Родители", "Опекунство", 
            "Село", "Иностранец", "КодФакультета", "Город_ПП", "Регион_ПП", "Где_Находится_УЗ", "Уч_Заведение", "reg_pp"]

In [239]:
drop_feat = [ "Статус"] # 'ID', "Код_группы",

In [240]:
feature_columns = list(train.columns.values)
for x in drop_feat:
    feature_columns.remove(x)

In [241]:
train[feature_columns]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,СрБаллАттестата,gape,gape_age,Страна_ПП_Уч_Заведение_diff,Страна_Родители_Уч_Заведение_diff,Город_ПП_Уч_Заведение_diff,Регион_ПП_Уч_Заведение_diff,Где_Находится_УЗ_Уч_Заведение_diff,reg_pp,Strange_disp
0,72716,16019,2015,Жен,ОО,Английский,23978,"МБОУ ""СОШ №59""",Барнаул,2014.0,...,4.294,1.0,211,0.000000,0.000000,0.111111,0.071429,0.071429,алтайский край,0.0
1,63306,14895,2010,Муж,ЦН,None,23908,none,none,-1.0,...,67.000,2011.0,221,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,алтайский край,0.0
2,82469,20103,2017,Жен,ДН,Английский,23991,МБОУ Алтайская СОШ №5,Алтайское,2017.0,...,4.133,0.0,222,0.166667,0.166667,0.636364,0.500000,0.380952,алтайский край,0.0
3,81661,19838,2017,Жен,БН,Английский,23940,ФГБОУ ВО Алтайский государственный университет,Барнаул,2017.0,...,74.000,0.0,273,0.333333,0.333333,0.200000,0.714286,0.384615,алтайский край,0.0
4,81509,19803,2017,Жен,БН,Английский,23946,ФГБОУ ВО Алтайский государственный университет,Барнаул,2017.0,...,53.000,0.0,267,0.333333,0.333333,0.200000,0.714286,0.384615,алтайский край,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13579,74283,17593,2016,Муж,ОО,Английский,23985,МБОУ СОШ №98,Барнаул,2016.0,...,59.000,0.0,216,0.000000,0.000000,0.142857,0.083333,0.083333,алтайский край,0.0
13580,71516,19433,2015,Жен,ОО,Немецкий,23973,"МКОУ ""Георгиевская СОШ""",Георгиевка,2015.0,...,82.000,0.0,216,0.166667,0.166667,0.100000,0.142857,0.347826,алтайский край,0.0
13581,76970,18723,2016,Муж,СН,Английский,23979,МБОУ СОШ №113 им. Сергея Семенова,Барнаул,2016.0,...,3.400,0.0,222,0.166667,0.166667,0.142857,0.071429,0.030303,алтайский край,0.0
13582,48548,14663,2012,Жен,ОО,None,23947,none,none,-1.0,...,95.000,2013.0,206,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,none,0.0


In [242]:
from lightgbm import LGBMClassifier

In [243]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [244]:
X_TRAIN = train[feature_columns].copy()
y = train["Статус"].values

In [245]:
for x in cat_feat:
    X_TRAIN[x] = X_TRAIN[x].astype("category")

In [246]:
X_TRAIN["f1"] = X_TRAIN["Код_группы"]//10000
X_TRAIN["f2"] = X_TRAIN["Код_группы"]//1000
X_TRAIN["f3"] = X_TRAIN["Код_группы"]//100
X_TRAIN["f4"] = X_TRAIN["Код_группы"]//10

X_TRAIN["l1"] = X_TRAIN["Код_группы"]%10000
X_TRAIN["l2"] = X_TRAIN["Код_группы"]%1000
X_TRAIN["l3"] = X_TRAIN["Код_группы"]%100
X_TRAIN["l4"] = X_TRAIN["Код_группы"]%10

X_TRAIN["df"] = X_TRAIN["Код_группы"]%2000
X_TRAIN["dl"] = X_TRAIN["Код_группы"]//2000

X_TRAIN = X_TRAIN.merge(work_count, on=name, how="left") #average='
X_TRAIN = X_TRAIN.merge(work_diff, on=name, how="left") #average='
X_TRAIN = X_TRAIN.merge(work_diff_inv, on=name, how="left") #average='
X_TRAIN = X_TRAIN.merge(dfid, on="ID", how="left") #average='
X_TRAIN = X_TRAIN.merge(dfidinv, on="ID", how="left") #average='

#X_TRAIN = X_TRAIN.merge(work_count, on=name, how="left") #average='macro'

In [247]:
X_TRAIN = pd.concat([X_TRAIN, uchzavdf], axis=1)

In [248]:
X_TRAIN["Status"] = y
X_TRAIN = X_TRAIN.sort_values(["Код_группы", "ID"])
y = X_TRAIN["Status"].values
X_TRAIN = X_TRAIN.drop(columns=["Status"])
X_TRAIN = X_TRAIN.reset_index()

In [249]:
X_TRAIN = X_TRAIN.drop(columns=["index"])

In [250]:
X_TEST = test[feature_columns].copy()

for x in cat_feat:
    X_TEST[x] = X_TEST[x].astype("category")
    
X_TEST["f1"] = X_TEST["Код_группы"]//10000
X_TEST["f2"] = X_TEST["Код_группы"]//1000
X_TEST["f3"] = X_TEST["Код_группы"]//100
X_TEST["f4"] = X_TEST["Код_группы"]//10

X_TEST["l1"] = X_TEST["Код_группы"]%10000
X_TEST["l2"] = X_TEST["Код_группы"]%1000
X_TEST["l3"] = X_TEST["Код_группы"]%100
X_TEST["l4"] = X_TEST["Код_группы"]%10

X_TEST["df"] = X_TEST["Код_группы"]%2000
X_TEST["dl"] = X_TEST["Код_группы"]//2000

X_TEST = X_TEST.merge(work_count, on=name, how="left")
X_TEST = X_TEST.merge(work_diff, on=name, how="left") #average='
X_TEST = X_TEST.merge(work_diff_inv, on=name, how="left") #average='
X_TEST = X_TEST.merge(dfid, on="ID", how="left") #average='
X_TEST = X_TEST.merge(dfidinv, on="ID", how="left") #average='


X_TEST = pd.concat([X_TEST, uchzavdf_test], axis=1)

In [251]:
def most_common(lst):
    return max(set(lst), key=lst.count)

def fit_lgbms(params, cat_feat, X, y):
    lgbms = []
    seeds = [154, 2277, 42, 5, 8713]
    for seed in seeds:
        
        lgbm = LGBMClassifier(is_unbalance=True, **params, random_state=seed)
        
        lgbm.fit(X, y, categorical_feature=cat_feat)
        
        lgbms.append(lgbm)
        
    return lgbms

def fit_lgbms_full(params, cat_feat, X, y):
    lgbms = []
    seeds = [154, 2277, 42, 5, 8713]
    for seed in seeds:
        
        lgbm = LGBMClassifier(class_weight="balanced", **params, random_state=seed)
        
        lgbm.fit(X, y, categorical_feature=cat_feat)
        
        lgbms.append(lgbm)
        
    return lgbms


def prdict_lgbms(lgbms, X):
    
    ys = []
    
    for lgbm in lgbms:
        
        y = lgbm.predict(X)
        ys.append(y)
        
    yf = []
    for i in range(len(X)):
        l = [ys[j][i] for j in range(len(lgbms))]
        yf.append(most_common(l))
           
    return yf


def prdict_lgbms_proba(lgbms, X):
    
    ys = []
    
    for lgbm in lgbms:
        
        y = lgbm.predict_proba(X)[:,1].reshape(1, -1)
        ys.append(y)
               
    return np.vstack(ys).mean(axis=0)

def prdict_lgbms_proba_full(lgbms, X):
    
    ys = []
    
    for lgbm in lgbms:
        
        y = lgbm.predict_proba(X) #[:,1].reshape(1, -1)
        ys.append(y)
               
    return np.stack(ys).mean(axis=0)

In [252]:
import pickle
with open('sm_params.pickle', 'rb') as handle:
    sm_params = pickle.load(handle)

In [253]:
def sm_bagged_fit_predict(X_TRAIN, y, X_TEST, sm_params, n_folds, seeds):
    ovo_test_list = []
    sm_train_list = []
       
    features_sm_test = []
        
    for seed in seeds:
        
        features_ovo = []

        preds = []
    

        for ti, di in StratifiedKFold(n_splits=n_folds, shuffle=True,  random_state=seed).split(X_TRAIN, y):

            Xt = X_TRAIN.iloc[ti]
            yt = y[ti]


            Xd = X_TRAIN.iloc[di]
            yd = y[di]

            lgbms = fit_lgbms_full(sm_params, cat_feat, Xt, yt)
            ypred = prdict_lgbms_proba_full(lgbms, Xd)
            features_sm_test.append(prdict_lgbms_proba_full(lgbms, X_TEST))

            preds.append((di, ypred))

                    
        
        yprom = np.zeros((len(X_TRAIN), 3))

        for i in range(n_folds):
            yprom[preds[i][0]] = preds[i][1]

        #features_ovo.append(yprom.copy())

        #ovo_test = [np.vstack(features_ovo_test[0]).mean(axis=0), np.vstack(features_ovo_test[1]).mean(axis=0), np.vstack(features_ovo_test[2]).mean(axis=0)]
        #ovo_test = np.vstack(ovo_test).T
        #ovo_test_list.append(ovo_test)
        
        #ovo_train = np.vstack(features_ovo).T
        sm_train_list.append(yprom)
    
    #print(features_sm_test)
    return np.stack(features_sm_test).mean(axis=0), np.stack(sm_train_list).mean(axis=0)

In [254]:
from sklearn.linear_model import LogisticRegression

In [255]:
X, Ct = sm_bagged_fit_predict(X_TRAIN, y, X_TEST, sm_params, 5, [333, 444, 555, 777, 888]) #[333, 444, 555, 777, 888]
Cd = X

C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.090931378259608, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.090931378259608


C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.090931378259608, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.090931378259608


C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


...

C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.090931378259608, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.090931378259608


C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.090931378259608, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.090931378259608


C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


In [256]:
ypred = np.array([-1, 3, 4])[np.argmax(Cd, axis=1)] #Cd

subm = pd.read_csv("sample_submission.csv")
subm["Статус"] = ypred
subm.to_csv("rez.csv", encoding='utf-8', sep=',', index=False)

In [257]:
yp = np.array([-1, 3, 4])[np.argmax(Ct, axis=1)] #Cd

In [258]:
f1_score(y, yp, average="macro")

0.8046060214720209

In [107]:
def ovo_bagged_fit_predict(X_TRAIN, y, X_TEST, ovo_params, n_folds, seeds):
    ovo_test_list = []
    ovo_train_list = []
       
    for seed in seeds:
    
        features_ovo_test = [[], [], []]
        
        features_ovo = []

        preds = [[], [], []]
    

        for ti, di in StratifiedKFold(n_splits=n_folds, shuffle=True, ).split(X_TRAIN, y):

            for i, pair in enumerate([(-1, 3, False), (-1, 4, False), (3, 4, True)]):

                Xt = X_TRAIN.iloc[ti]
                yt = y[ti]

                Xt = Xt[(yt==pair[0])|(yt==pair[1])]
                yt = yt[(yt==pair[0])|(yt==pair[1])]

                yt = (yt==pair[0]).astype(np.int32)



                Xd = X_TRAIN.iloc[di]
                yd = y[di]

                yd = (yd==pair[0]).astype(np.int32)


                if(pair[2]):

                    lgbms = fit_lgbms(ovo_params[i], cat_feat, Xt, yt)
                    ypred = prdict_lgbms_proba(lgbms, Xd)
                    features_ovo_test[i].append(prdict_lgbms_proba(lgbms, X_TEST))

                else:
                    lgbm = LGBMClassifier(is_unbalance=True, **ovo_params[i])
                    

                    lgbm.fit(
                        Xt,
                        yt, categorical_feature=cat_feat
                    )
                    
                    ypred = lgbm.predict_proba(Xd)[:,1]
                
                    features_ovo_test[i].append(lgbm.predict_proba(X_TEST)[:,1])
                preds[i].append((di, ypred))

                    
        for j in range(3):

            yprom = np.zeros((len(X_TRAIN), ))

            for i in range(n_folds):
                yprom[preds[j][i][0]] = preds[j][i][1]

            features_ovo.append(yprom.copy())

        ovo_test = [np.vstack(features_ovo_test[0]).mean(axis=0), np.vstack(features_ovo_test[1]).mean(axis=0), np.vstack(features_ovo_test[2]).mean(axis=0)]
        ovo_test = np.vstack(ovo_test).T
        ovo_test_list.append(ovo_test)
        
        ovo_train = np.vstack(features_ovo).T
        ovo_train_list.append(ovo_train)
    
    
    return np.stack(ovo_test_list).mean(axis=0), np.stack(ovo_train_list).mean(axis=0)


In [119]:
with open('ovo_params.pickle', 'rb') as handle:
    ovo_params = pickle.load(handle)

X, At = ovo_bagged_fit_predict(X_TRAIN, y, X_TEST, ovo_params, 5, [333, 444, 555, 777, 888]) #[i for i in range(77,77*11,77)]
Ad = X

C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.08170754256230839, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.08170754256230839


C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.04559274877300563, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.04559274877300563


C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


...

C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.04559274877300563, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.04559274877300563


C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.05057939553441975, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.05057939553441975


C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Program Files\Python38\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


In [120]:
def calculate_ovo_score(X):
    votes = np.zeros((X.shape[0], 3))
    sum_of_conf = np.zeros((X.shape[0], 3))

    votes[X[:,0]>0.5, 0] += 1
    votes[X[:,0]<=0.5, 1] += 1

    votes[X[:,1]>0.5, 0] += 1
    votes[X[:,1]<=0.5, 2] += 1

    votes[X[:,2]>0.5, 1] += 1
    votes[X[:,2]<=0.5, 2] += 1

    sum_of_conf[:, 0] -= X[:,0]
    sum_of_conf[:, 1] += X[:,0]

    sum_of_conf[:, 0] -= X[:,1]
    sum_of_conf[:, 2] += X[:,1]

    sum_of_conf[:, 1] -= X[:,2]
    sum_of_conf[:, 2] += X[:,2]

    transformed_confidences = sum_of_conf / (
        3 * (np.abs(sum_of_conf) + 1)
    )
    return votes + transformed_confidences

In [121]:
yp = np.array([-1, 3, 4])[np.argmax(calculate_ovo_score(At), axis=1)] #Cd
f1_score(y, yp, average="macro")

0.8044879375630695

In [135]:
def transform2metafeat(A, C):
    X = A
    votes = np.zeros((X.shape[0], 3))
    sum_of_conf = np.zeros((X.shape[0], 3))

    votes[X[:,0]>0.5, 0] += 1
    votes[X[:,0]<=0.5, 1] += 1

    votes[X[:,1]>0.5, 0] += 1
    votes[X[:,1]<=0.5, 2] += 1

    votes[X[:,2]>0.5, 1] += 1
    votes[X[:,2]<=0.5, 2] += 1

    sum_of_conf[:, 0] -= X[:,0]
    sum_of_conf[:, 1] += X[:,0]

    sum_of_conf[:, 0] -= X[:,1]
    sum_of_conf[:, 2] += X[:,1]

    sum_of_conf[:, 1] -= X[:,2]
    sum_of_conf[:, 2] += X[:,2]

    transformed_confidences = sum_of_conf / (
        3 * (np.abs(sum_of_conf) + 1)
    )
        
    Am = np.zeros((A.shape[0],3))
    for i, j in zip(np.arange(A.shape[0]), (votes+transformed_confidences).argmax(axis=1)):
        Am[i, j] = 1
        
    Cm = np.zeros((C.shape[0],3))
    for i, j in zip(np.arange(A.shape[0]), C.argmax(axis=1)):
        Cm[i, j] = 1
    
    
    
    #return np.hstack([B, Am, Bm,
    #    X, votes, transformed_confidences , votes+transformed_confidences, C])
    return np.hstack([C, Cm, Am[:,:], X[:,:]])

In [151]:
mft = transform2metafeat(At, Ct)
mfd = transform2metafeat(Ad, Cd)

model=LogisticRegression(class_weight="balanced", multi_class = 'multinomial', max_iter=1000, C=0.000001) #MLPClassifier(hidden_layer_sizes=(20,), alpha=0.1) #ExtraTreesClassifier(n_estimators=2000, criterion='entropy', max_depth=None,  min_samples_leaf=100)

model.fit(mft, y)

ypred = model.predict(mfd)


In [152]:
subm = pd.read_csv("sample_submission.csv")
subm["Статус"] = ypred
subm.to_csv("rez_stacked.csv", encoding='utf-8', sep=',', index=False)

In [153]:
import pandas as pd
subm_an = pd.read_csv("rez_stacked.csv")
subm = pd.read_csv("rez.csv")
a = subm_an["Статус"].values
b = subm["Статус"].values
from sklearn.metrics import confusion_matrix
confusion_matrix(a, b)

array([[ 349,   39,    1],
       [   0, 2057,    0],
       [   3,   44, 4198]], dtype=int64)

array([[2.22376577e-01, 6.04549150e-04, 3.96270647e-03],
       [6.56151020e-02, 1.90558625e-03, 1.61825770e-03],
       [6.50833409e-01, 1.23668699e-01, 6.32549453e-01],
       ...,
       [5.72259246e-02, 1.09015090e-03, 1.09331964e-02],
       [9.50506794e-03, 8.85701288e-05, 6.85858052e-02],
       [2.97426134e-03, 2.97520976e-05, 9.49939080e-01]])